In [84]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

# JIRA API credentials
base_url = "https://corecard.atlassian.net/"
api_token = ""
email = ""

# API endpoint to fetch issues
endpoint = f"{base_url}rest/api/3/search"
headers = {
    "Accept": "application/json"
}

# JQL query
jql = "project = CCP AND issuetype = epic AND fixVersion = R40 AND status not in (Cancelled)"
params = {
    "jql": jql,
    "maxResults": 100
}

# Request
try:
    response = requests.get(
        endpoint,
        headers=headers,
        params=params,
        auth=HTTPBasicAuth(email, api_token)
    )

    if response.status_code == 200:
        data = response.json()

        # Extracting issues from the response
        issues = data.get('issues', [])

        # Preparing a list to store data for the DataFrame
        df_data = []

        # Iterating over each issue and extracting necessary information from the JSON file
        for issue in issues:
            key = issue.get('key', '')
            fields = issue.get('fields', {})

            # Extracting the required fields
            summary = fields.get('summary', '')
            status = fields.get('status', {})
            status_name = status.get('name', '') if status else ''
            assignee = fields.get('assignee', {})
            assignee_name = assignee.get('displayName', 'Unassigned') if assignee else 'Unassigned'
            created = fields.get('created', '-')
            fix_versions = fields.get('fixVersions', [])
            fix_versions_names = [version.get('name', '') for version in fix_versions]
            fix_versions_str = ', '.join(fix_versions_names)
            parent = fields.get('parent', {})
            parent_key = parent.get('key', '-')
            # Custom fields
            ba = fields.get('customfield_10112', [])
            ba_manager = ba[0].get('displayName') if ba else None
            ba_res = fields.get('customfield_10113', [])
            ba_resource = ba_res[0].get('displayName') if ba_res else None
            dev = fields.get('customfield_10114', [])
            dev_manager = dev[0].get('displayName') if dev else None
            dev_resource = fields.get('customfield_10115', [])
            development_resource = dev_resource[0].get('displayName') if dev_resource else None
            baeffort = fields.get('customfield_10122', '-')
            ba_effort = baeffort
            qaeffort = fields.get('customfield_10125', '-')
            qa_effort = qaeffort

            # Appending the extracted information as a dictionary to the list
            df_data.append({
                'Key': key,
                'Summary': summary,
                'Status': status_name,
                'Assignee': assignee_name,
                'Created': created,
                'Fix Version': fix_versions_str,
                'Parent Key': parent_key,
                'BA Manager': ba_manager,
                'BA Resource': ba_resource,
                'Dev Manager': dev_manager,
                'Dev Resource': development_resource,
                'BA Effort': ba_effort,
                'QA Effort': qa_effort
            })

        # Creating a Master DataFrame from the extracted data
        df = pd.DataFrame(df_data)

        # Function to make the Key column clickable
        def make_key_clickable(df):
            # Making the Key column a clickable hyperlink
            df['Key'] = df['Key'].apply(lambda x: f'<a href="{base_url}browse/{x}" target="_blank">{x}</a>')
            return df

        # Method to process the DataFrame based on field_id:
        def segregate_dataframe(df, field_id):
            if field_id == 'customfield_10112':  # BA Manager field_id
                df_filtered = df[df['BA Manager'].isna()]
                df_filtered = make_key_clickable(df_filtered)  # Converting 'Key' column to clickable
                return df_filtered[['Key', 'Summary', 'Status', 'Assignee', 'Created', 'Fix Version', 'Parent Key', 'BA Manager', 'BA Resource', 'BA Effort']]
            elif field_id == 'customfield_10114':  # Dev Manager field_id
                df_filtered = df[df['Dev Manager'].isna()]
                df_filtered = make_key_clickable(df_filtered) 
                return df_filtered[['Key', 'Summary', 'Status', 'Assignee', 'Created', 'Fix Version', 'Parent Key', 'Dev Manager', 'Dev Resource']]
            else:
                raise ValueError("Invalid field_id provided")

        # Get DataFrames for both BA Manager and Dev Manager
        df_ba_manager = segregate_dataframe(df, 'customfield_10112')
        df_dev_manager = segregate_dataframe(df, 'customfield_10114')

        # Convert DataFrames to HTML tables
        df_ba_manager_html = df_ba_manager.to_html(index=False, escape=False)  # escape=False to allow HTML in cells
        df_dev_manager_html = df_dev_manager.to_html(index=False, escape=False) 

        # Email configurations
        smtp_server = "corecard-com.mail.protection.outlook.com"
        smtp_port = 25
        smtp_user = 'sanidhya.mitra@corecard.com'

        # Recipient and CC
        recipient_emails = {
            "pawan.linjhara@corecard.com": df_ba_manager_html, #put in ba
            "kalyani.kharate@corecard.com": df_dev_manager_html #put in development
        }
        cc_email = "sanidhya.mitra@corecard.com"

        # Function to send email
        def send_email(subject, body, to_email, cc_emails):
            msg = MIMEMultipart()
            msg['From'] = smtp_user
            msg['To'] = to_email
            msg['Cc'] = cc_email  # Adding CC emails here
            msg['Subject'] = 'This is test 2'

            # Attaching the main message body:
            msg.attach(MIMEText(body, 'html'))

            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.sendmail(smtp_user, [to_email] + cc_emails, msg.as_string())
                    print(f"Email sent to {to_email} with CC to {cc_emails}")
            except Exception as e:
                print(f"Failed to send email to {to_email}. Error: {e}")

        # Send emails
        for email, html_content in recipient_emails.items():
            send_email("Example test 2", html_content, email, [cc_email])

    else:
        print(f"Failed to fetch data from JIRA API. Status Code: {response.status_code}")
        print(f"Response: {response.text}")

except Exception as e:
    print(f"An error occurred: {e}")


C:\Users\Sanidhya.Mitra\AppData\Local\Temp\ipykernel_1236\861928966.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key'] = df['Key'].apply(lambda x: f'<a href="{base_url}browse/{x}" target="_blank">{x}</a>')
C:\Users\Sanidhya.Mitra\AppData\Local\Temp\ipykernel_1236\861928966.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key'] = df['Key'].apply(lambda x: f'<a href="{base_url}browse/{x}" target="_blank">{x}</a>')


Email sent to pawan.linjhara@corecard.com with CC to ['sanidhya.mitra@corecard.com']
Email sent to kalyani.kharate@corecard.com with CC to ['sanidhya.mitra@corecard.com']
